# Data conversion

## 1. Get started

In [1]:
resource_group = "kainos-schools-outreach"
region = "uksouth"

In [2]:
service_name = "pdf-form-recogniser"

### 1.1 Create Form Recogniser service resource

The following commands create the service and get the endpoint and key for use in the SDK

In [ ]:
!az cognitiveservices account create -n $service_name -g $resource_group -l $region --kind FormRecognizer --sku F0 --yes

In [3]:
endpoint = !az cognitiveservices account show -n $service_name -g $resource_group --query "properties.endpoint" | cut -d \" -f2

In [4]:
key = !az cognitiveservices account keys list -n $service_name -g $resource_group --query "key1" | cut -d \" -f2

### 1.2 Install Python SDK for Form Recogniser

In [ ]:
!pip install azure-ai-formrecognizer==3.2.0

In [5]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
client = DocumentAnalysisClient(endpoint=endpoint[0], credential=AzureKeyCredential(key[0]))

In [6]:
def result_kv_pairs_to_df(result):
    keys = []
    vals = []
    for kv_pair in result.key_value_pairs:
        keys.append("" if not kv_pair.key else kv_pair.key.content)
        vals.append("" if not kv_pair.value else kv_pair.value.content)
    return pd.DataFrame({"Keys": keys, "Values": vals})

In [7]:
def result_tables_to_df(result):
    dfs = []
    for table in result.tables:
        
        table_arr = np.empty([table.row_count, table.column_count], dtype=object)
        for cell in table.cells:
            table_arr[cell.row_index, cell.column_index] = cell.content
        
        dfs.append(pd.DataFrame(table_arr))
    return dfs

In [3]:
import numpy as np
import pandas as pd
import glob
from natsort import natsorted

## Dataset 2

In [11]:
fns = natsorted(glob.glob("raw/dataset2/dataset*.pdf"))
pollers = [client.begin_analyze_document("prebuilt-document", open(fn, "rb")) for fn in fns]
results = [poller.result() for poller in pollers]

In [12]:
dfs = []
for result in results:
    for df in result_tables_to_df(result):
        dfs.append(df)

In [14]:
year11 = dfs[0]
year12 = pd.concat([dfs[1], dfs[2]], ignore_index=True)
year13 = dfs[3]
year14 = dfs[4]

In [15]:
year11.columns = year11.iloc[0]
year11 = year11.drop(year11.index[0]).reset_index(drop=True)

In [16]:
year12.columns = year12.iloc[0]
year12 = year12.drop(year12.index[0]).reset_index(drop=True)

In [17]:
year13.columns = year13.iloc[0]
year13 = year13.drop(year13.index[0]).reset_index(drop=True)

In [18]:
year14.columns = year14.iloc[1]
year14 = year14.drop(year14.index[0:2]).reset_index(drop=True)

In [19]:
years = pd.concat([year11, year12, year13, year14], keys=["11", "12", "13", "14"]).reset_index(names=["year", "orig_index"]).drop(columns=["orig_index"])

In [20]:
years["subject"] = "Computer Studies"

In [22]:
years.to_csv("csv/dataset2_dirty.csv")

## Dataset 1

In [23]:
fns = natsorted(glob.glob("raw/dataset1/dataset*.pdf"))
pollers = [client.begin_analyze_document("prebuilt-document", open(fn, "rb")) for fn in fns]
results = [poller.result() for poller in pollers]

In [24]:
dfs = list(map(result_tables_to_df, results))

In [ ]:
for i,dfs_page in enumerate(dfs):
    print("PAGE ", i+1)
    for df in dfs_page:
        display(df.head())

In [55]:
year11 = pd.concat([dfs[0][0], dfs[1][0], dfs[2][0], dfs[3][0], dfs[4][0]], ignore_index=True)
year12 = pd.concat([dfs[4][1], dfs[5][0], dfs[6][0], dfs[7][0], dfs[8][0]], ignore_index=True)
year13 = pd.concat([dfs[8][1], dfs[9][0], dfs[10][0], dfs[11][0], dfs[12][0]], ignore_index=True)
year14 = pd.concat([dfs[12][1], dfs[13][0], dfs[14][0], dfs[15][0], dfs[16][0], dfs[17][0]], ignore_index=True)

In [56]:
year11.columns = year11.iloc[0]
year11 = year11.drop(year11.index[0]).reset_index(drop=True)

In [57]:
year12.columns = year12.iloc[4]
year12 = year12.drop(year12.index[0:5]).reset_index(drop=True)

In [58]:
year13.columns = year13.iloc[3]
year13 = year13.drop(year13.index[0:4]).reset_index(drop=True)
year13 = year13.drop(year13.index[137:]).reset_index(drop=True)

In [59]:
year14 = pd.concat([dfs[12][0][24:], year14], ignore_index=True)

In [60]:
year14.columns = year14.iloc[0]
year14 = year14.drop(year14.index[0]).reset_index(drop=True)
year14 = year14.drop(year14.index[1:3]).reset_index(drop=True)

In [63]:
years = pd.concat([year11, year12, year13, year14], keys=["11", "12", "13", "14"]).reset_index(names=["year", "orig_index"]).drop(columns=["orig_index"])

In [64]:
years["subject"] = "ICT"

In [65]:
years.to_csv("csv/dataset1_dirty.csv")

## Merge datasets

In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
df1 = pd.read_csv("csv/dataset1_dirty.csv", index_col=0)
df2 = pd.read_csv("csv/dataset2_dirty.csv", index_col=0)

In [3]:
df = pd.concat([df1, df2], ignore_index=True)

In [4]:
def clean_strings(x):
    return x.replace("\n", "").replace(":selected:", "").replace(":unselected:", "").strip() if isinstance(x, str) else x

In [5]:
def get_course_code(cnacc):
    if not isinstance(cnacc, str): return cnacc
    result = re.split(r"\(([^\(]+)\)$", cnacc)
    return result[0] if len(result) == 1 else (result[0], result[1])

In [6]:
df = df.applymap(clean_strings).applymap(get_course_code)

In [7]:
df[['course_name', 'course_code']] = pd.DataFrame(df["Course Name and Course Code"].tolist(), index=df.index)
df[["School", "school_id"]] = pd.DataFrame(df["School"].tolist(), index=df.index)

In [8]:
df = df.drop(columns=["Course Name and Course Code"]).applymap(clean_strings).astype(
{"Pupils": int, "school_id": int})

In [9]:
df.to_csv("csv/dataset1_2.csv", index=False)

In [10]:
df

,year,School,Label,AOL,Pupils,Provider,subject,course_name,course_code,school_id
0,11,St Patrick's Academy (Lisburn),A,LLW,15,Own School,ICT,OCN NI Level 2 Certificate in Information Tech...,601/8497/8,4230165
1,11,Larne High School,A,LLW,35,Own School,ICT,OCN NI Level 2 Certificate in Information Tech...,601/8497/8,3210038
2,11,Cullybackey College,A,LLW,15,Own School,ICT,OCN NI Level 2 Certificate in Information Tech...,601/8497/8,3210172
3,11,St Mary's High (Newry),A,LLW,17,Own School,ICT,OCN NI Level 2 Certificate in Information Tech...,601/8497/8,5230108
4,11,St Colman's High School,A,LLW,47,Own School,ICT,OCN NI Level 2 Certificate in Information Tech...,601/8497/8,4230161
...,...,...,...,...,...,...,...,...,...,...
553,14,Campbell College,G,ST,4,Collaboration with other school,Computer Studies,WJEC Level 3 Advanced GCE in Computer Science,601/5345/3,1420020
554,14,Royal Belfast Academical Institution,G,ST,13,Own School,Computer Studies,WJEC Level 3 Advanced GCE in Computer Science,601/5345/3,1420027
555,14,Belfast High School,G,ST,5,Own School,Computer Studies,WJEC Level 3 Advanced GCE in Computer Science,601/5345/3,3420077
556,14,Strathearn School,G,ST,1,Collaboration with other school,Computer Studies,WJEC Level 3 Advanced GCE in Computer Science,601/5345/3,1420089


In [11]:
for colname in df.columns:
    display(df.groupby(colname).count())

,School,Label,AOL,Pupils,Provider,subject,course_name,course_code,school_id
year,,,,,,,,,
11,131,131,131,131,131,131,131,131,131
12,126,126,126,126,126,126,126,126,126
13,153,153,153,153,153,153,153,153,153
14,148,148,148,148,148,148,148,148,148


,year,Label,AOL,Pupils,Provider,subject,course_name,course_code,school_id
School,,,,,,,,,
Abbey Community College,5,5,5,5,5,5,5,5,5
All Saints College,4,4,4,4,4,4,4,4,4
Antrim Grammar School,6,6,6,6,6,6,6,6,6
Aquinas Grammar,1,1,1,1,1,1,1,1,1
Ashfield Boys' High School,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...
Sullivan Upper School,4,4,4,4,4,4,4,4,4
Ulidia Integrated College,4,4,4,4,4,4,4,4,4
Victoria College,2,2,2,2,2,2,2,2,2


,year,School,AOL,Pupils,Provider,subject,course_name,course_code,school_id
Label,,,,,,,,,
A,523,523,523,523,523,523,523,523,523
G,35,35,35,35,35,35,35,35,35


,year,School,Label,Pupils,Provider,subject,course_name,course_code,school_id
AOL,,,,,,,,,
ES,1,1,1,1,1,1,1,1,1
LLW,136,136,136,136,136,136,136,136,136
ST,421,421,421,421,421,421,421,421,421


,year,School,Label,AOL,Provider,subject,course_name,course_code,school_id
Pupils,,,,,,,,,
1,14,14,14,14,14,14,14,14,14
2,7,7,7,7,7,7,7,7,7
3,7,7,7,7,7,7,7,7,7
4,17,17,17,17,17,17,17,17,17
5,12,12,12,12,12,12,12,12,12
...,...,...,...,...,...,...,...,...,...
129,1,1,1,1,1,1,1,1,1
132,1,1,1,1,1,1,1,1,1
140,1,1,1,1,1,1,1,1,1


,year,School,Label,AOL,Pupils,subject,course_name,course_code,school_id
Provider,,,,,,,,,
Collaboration with other school,46,46,46,46,46,46,46,46,46
FE,3,3,3,3,3,3,3,3,3
Own School,509,509,509,509,509,509,509,509,509


,year,School,Label,AOL,Pupils,Provider,course_name,course_code,school_id
subject,,,,,,,,,
Computer Studies,68,68,68,68,68,68,68,68,68
ICT,490,490,490,490,490,490,490,490,490


,year,School,Label,AOL,Pupils,Provider,subject,course_code,school_id
course_name,,,,,,,,,
AQA Level 1 Level 2 GCSE (9-1) in Computer Science,5,5,5,5,5,5,5,5,5
AQA Level 3 Advanced GCE in Computer Science,6,6,6,6,6,6,6,6,6
CCEA Level 3 Advanced GCE in Software Systems Development,28,28,28,28,28,28,28,28,28
CCEA Level 3 Advanced Subsidiary GCE in Software Systems Development,31,31,31,31,31,31,31,31,31
OCN NI Level 2 Certificate in Information Technology Applications,130,130,130,130,130,130,130,130,130
OCN NI Level 3 Diploma in Information Technology Applications,6,6,6,6,6,6,6,6,6
OCR Level 1 Level 2 GCSE (9-1) in Computer Science,21,21,21,21,21,21,21,21,21
OCR Level 2 Extended Certificate in IT User Skills (ITQ) (QCF),1,1,1,1,1,1,1,1,1
OCR Level 2 ITQ Certificate in IT User Skills (QCF),3,3,3,3,3,3,3,3,3


,year,School,Label,AOL,Pupils,Provider,subject,course_name,school_id
course_code,,,,,,,,,
500/6743/6,3,3,3,3,3,3,3,3,3
500/7832/X,7,7,7,7,7,7,7,7,7
500/9147/5,14,14,14,14,14,14,14,14,14
500/9150/5,1,1,1,1,1,1,1,1,1
600/1062/9,1,1,1,1,1,1,1,1,1
600/4231/X,3,3,3,3,3,3,3,3,3
600/4237/0,1,1,1,1,1,1,1,1,1
600/4623/5,76,76,76,76,76,76,76,76,76
600/4789/6,73,73,73,73,73,73,73,73,73


,year,School,Label,AOL,Pupils,Provider,subject,course_name,course_code
school_id,,,,,,,,,
1210014,4,4,4,4,4,4,4,4,4
1210015,4,4,4,4,4,4,4,4,4
1210021,5,5,5,5,5,5,5,5,5
1210022,6,6,6,6,6,6,6,6,6
1230053,6,6,6,6,6,6,6,6,6
...,...,...,...,...,...,...,...,...,...
5420260,4,4,4,4,4,4,4,4,4
5420263,2,2,2,2,2,2,2,2,2
5420268,6,6,6,6,6,6,6,6,6


## 3. Clean up

Delete resource. After deletion, resource can be restored but new resource cannot be made with same name. Purge to fully delete.

In [ ]:
!az cognitiveservices account delete -n $service_name -g $resource_group

In [ ]:
!az cognitiveservices account purge -n $service_name -g $resource_group -l $region